In [13]:
import sys
import time
import unittest

import rospy
import actionlib
from control_msgs.msg import FollowJointTrajectoryAction, FollowJointTrajectoryGoal, FollowJointTrajectoryResult
from ur_dashboard_msgs.msg import SetModeAction, SetModeGoal, RobotMode
from std_srvs.srv import Trigger, TriggerRequest
from trajectory_msgs.msg import JointTrajectoryPoint

import numpy as np

In [2]:
PKG = 'ur_robot_driver'
NAME = 'trajectory_test'

In [3]:
rospy.init_node('trajectory_testing_client')

In [4]:
client = actionlib.SimpleActionClient(
            '/scaled_pos_joint_traj_controller/follow_joint_trajectory', FollowJointTrajectoryAction)

In [5]:
timeout = rospy.Duration(3)

In [6]:
client.wait_for_server(timeout)

True

In [9]:
def init_robot():
    """Make sure the robot is booted and ready to receive commands"""
    mode_client = actionlib.SimpleActionClient(
        '/ur_hardware_interface/set_mode', SetModeAction)
    timeout = rospy.Duration(3)
    mode_client.wait_for_server(timeout)
    goal = SetModeGoal()
    goal.target_robot_mode = RobotMode.RUNNING
    goal.play_program = False # we use headless mode during tests

    mode_client.send_goal(goal)
    mode_client.wait_for_result()

    print("[mode_client]", mode_client.get_result().success)

    #send_program_srv = rospy.ServiceProxy("/ur_hardware_interface/resend_robot_program", Trigger)
    #send_program_srv.call()
    rospy.sleep(5)

In [10]:
init_robot()

('[mode_client]', True)


In [11]:
point = JointTrajectoryPoint()

In [12]:
point.positions

[]

In [14]:
goal = FollowJointTrajectoryGoal()

goal.trajectory.joint_names = ["elbow_joint", "shoulder_lift_joint", "shoulder_pan_joint",
                               "wrist_1_joint", "wrist_2_joint", "wrist_3_joint"]
position_list = [[-2.53875, -2.05062 + np.pi/24, 1.47606, -0.97761, -1.58473, 0.06858]]
duration_list = [6.0]

for i, position in enumerate(position_list):
    point = JointTrajectoryPoint()
    point.positions = position
    point.time_from_start = rospy.Duration(duration_list[i])
    goal.trajectory.points.append(point)

rospy.loginfo("Sending simple goal")

[INFO] [1606830387.943939]: Sending simple goal


In [ ]:
client.send_goal(goal)
client.wait_for_result()